
# Example: Intra-Subject Variability of ERP


In [ ]:
# Authors: Simon Kojima <simon.kojima@inria.fr>
#
# License: BSD (3-clause)

In [ ]:
import mne
import nearby
from moabb.datasets import Kojima2024B

In [ ]:
"""
Extract Epochs
"""

subject = 1
l_freq = 1
h_freq = 40
resample = 128
tmin = -0.1
tmax = 1.2
baseline = [-0.05, 0.0]

dataset = Kojima2024B(task="2stream")

data = dataset.get_data(subjects=[subject])

raws = list(data[subject]["0"].values())

for raw in raws:
    raw.filter(
        l_freq=l_freq,
        h_freq=h_freq,
        method="iir",
        iir_params={"ftype": "butter", "btype": "bandpass", "order": 4},
        phase="zero",
    )

raw = mne.concatenate_raws(raws)

raw.pick(picks="eeg")

epochs = mne.Epochs(raw, baseline=None, tmin=tmin, tmax=tmax)

epochs.load_data()

epochs.resample(resample)

epochs = epochs.apply_baseline(baseline=baseline)

In [ ]:
"""
# Within-Trial Temporal (WTTemp)
"""
wt_temp = nearby.metrics.within_trial_temporal(
    epochs["Target"],
    tmin=tmin,
    tmax=tmax,
    picks=["Cz", "CPz", "Pz"],
)

print(wt_temp)

exit()

In [ ]:
"""
# ATTV
"""
attv = nearby.metrics.attv(
    epochs["Target"],
    tmin=tmin,
    tmax=tmax,
    picks=["Cz", "CPz", "Pz"],
)

attv = attv["attv"].mean()

In [ ]:
"""
# ATSVA
"""
atsva = nearby.metrics.atsv(
    epochs["Target"],
    tmin=tmin,
    tmax=tmax,
    metrics="angle",
)

atsva = atsva["atsva"].mean()

In [ ]:
"""
Print results
"""

print("# Variability Metrics")
print(f" - ME    : {me:10.6f}")
print(f" - SMEAT : {smeat:10.6f}")
print(f" - WTTV  : {wttv:10.6f}")
print(f" - ATTV  : {attv:10.6f}")
print(f" - ATSVA : {atsva:10.6f}")